# Step 1: Preprocess RXJ1131 Data

目标：
- 以 `(x=2116, y=3377)` 为中心裁切 `200x200`
- 用四个 corner 的 `5x5` 像素估计背景噪声（MAD）
- 生成一个双扩展 FITS：第 1 张数据（SCI），第 2 张误差（ERR）

In [1]:
from pathlib import Path
import numpy as np
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.wcs import WCS

# ---- Config ----
DATA_DIR = Path('../../Data/RXJ1131')
INPUT_FITS = DATA_DIR / 'j8oi74010_drc.fits'
X_CENTER, Y_CENTER = 2116, 3377
CUT_SIZE = (200, 200)  # (ny, nx)
CORNER_N = 5
OUTPUT_FITS = DATA_DIR / f'j8oi74010_drc_cut_x{X_CENTER}_y{Y_CENTER}_200_scierr_corner{CORNER_N}.fits'

print('INPUT_FITS =', INPUT_FITS)
print('OUTPUT_FITS =', OUTPUT_FITS)

INPUT_FITS = ../../Data/RXJ1131/j8oi74010_drc.fits
OUTPUT_FITS = ../../Data/RXJ1131/j8oi74010_drc_cut_x2116_y3377_200_scierr_corner5.fits


In [2]:
with fits.open(INPUT_FITS, memmap=True) as hdul:
    sci_hdu = hdul['SCI']
    wht_hdu = hdul['WHT']

    sci = sci_hdu.data.astype(np.float64)
    wht = wht_hdu.data.astype(np.float64)

    cut_sci = Cutout2D(sci, position=(X_CENTER, Y_CENTER), size=CUT_SIZE, wcs=WCS(sci_hdu.header), mode='strict')
    cut_wht = Cutout2D(wht, position=(X_CENTER, Y_CENTER), size=CUT_SIZE, wcs=WCS(wht_hdu.header), mode='strict')

    sci_cut = cut_sci.data
    exp_eff = cut_wht.data

# ---- Background sigma from 4 corners, each CORNER_N x CORNER_N ----
corners = [
    sci_cut[:CORNER_N, :CORNER_N],
    sci_cut[:CORNER_N, -CORNER_N:],
    sci_cut[-CORNER_N:, :CORNER_N],
    sci_cut[-CORNER_N:, -CORNER_N:],
]
bg_pixels = np.concatenate([c.ravel() for c in corners])
bg_pixels = bg_pixels[np.isfinite(bg_pixels)]
if bg_pixels.size == 0:
    raise RuntimeError('No finite corner pixels for background estimation.')

bg_median = np.median(bg_pixels)
bg_mad = np.median(np.abs(bg_pixels - bg_median))
bg_sigma = 1.4826 * bg_mad

# ---- Error model ----
# SCI unit is e-/s. For EXP-type WHT (effective exposure time map):
# var_poisson_rate = max(SCI,0) / WHT
var_poisson_rate = np.zeros_like(sci_cut, dtype=np.float64)
good = np.isfinite(sci_cut) & np.isfinite(exp_eff) & (exp_eff > 0)
var_poisson_rate[good] = np.clip(sci_cut[good], 0.0, None) / exp_eff[good]

err_cut = np.full_like(sci_cut, np.nan, dtype=np.float64)
err_cut[good] = np.sqrt(var_poisson_rate[good] + bg_sigma**2)

# ---- Save one FITS with two image extensions ----
sci_hdr = sci_hdu.header.copy()
sci_hdr.update(cut_sci.wcs.to_header(relax=True))
sci_hdr['EXTNAME'] = 'SCI'
sci_hdr['BKG_SIG'] = (float(bg_sigma), 'Background sigma from 4x(5x5) corner pixels [e-/s]')
sci_hdr['BKG_MED'] = (float(bg_median), 'Background median from corner pixels [e-/s]')
sci_hdr['CUTX'] = (X_CENTER, 'Cutout center x (0-based pixel index)')
sci_hdr['CUTY'] = (Y_CENTER, 'Cutout center y (0-based pixel index)')
sci_hdr['CUTSIZE'] = (200, 'Cutout size in pixels')

err_hdr = sci_hdu.header.copy()
err_hdr.update(cut_sci.wcs.to_header(relax=True))
err_hdr['EXTNAME'] = 'ERR'
err_hdr['BUNIT'] = 'ELECTRONS/S'
err_hdr['BKG_SIG'] = (float(bg_sigma), 'Background sigma from 4x(5x5) corner pixels [e-/s]')
err_hdr['ERRMOD'] = 'SQRT( MAX(SCI,0)/WHT + BKG_SIG^2 )'
err_hdr['CUTX'] = (X_CENTER, 'Cutout center x (0-based pixel index)')
err_hdr['CUTY'] = (Y_CENTER, 'Cutout center y (0-based pixel index)')
err_hdr['CUTSIZE'] = (200, 'Cutout size in pixels')

hdul_out = fits.HDUList([
    fits.PrimaryHDU(data=sci_cut.astype(np.float32), header=sci_hdr),
    fits.ImageHDU(data=err_cut.astype(np.float32), header=err_hdr),
])
hdul_out.writeto(OUTPUT_FITS, overwrite=True)

print('Saved:', OUTPUT_FITS)
print('SCI shape:', sci_cut.shape)
print('ERR shape:', err_cut.shape)
print('Background median [e-/s]:', float(bg_median))
print('Background sigma  [e-/s]:', float(bg_sigma))

Saved: ../../Data/RXJ1131/j8oi74010_drc_cut_x2116_y3377_200_scierr_corner5.fits
SCI shape: (200, 200)
ERR shape: (200, 200)
Background median [e-/s]: 0.006205336423590779
Background sigma  [e-/s]: 0.00583716698703356
